In [1]:
from snowflake.snowpark import Session
import os

In [2]:
# Create Session object
connection_parameters = {
    "user":"p.arumugam@mindtree.com",
    "password":"Fast@2023",
    "account":"TTB91059.us-west-2",
    "role":"MIM_ROLE",
    "warehouse":"MIM_WH",
    "database":"D_FIN_DMA",
    "schema":"FIN_DMA"
}




In [3]:
test_session = Session.builder.configs(connection_parameters).create()
print(test_session.sql("select * from D_FIN_LND.FIN_WBG.CLIENT_DTA").collect())
print(test_session.sql("select current_warehouse(), current_database(), current_schema()").collect())

[Row(CLIENT_NAMES='MOBIKWdK', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='XPRESkBEEj', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='EURONEvGPyY', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='cLACqBS PcT LTD', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='SHRy SxI AxPLIANCES', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='ICICIGPoYSPOTRfCHAuiE', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='VgIwHAV SUDAn GHUGE', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='CRED', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CLIENT_NAMES='oDHARSH x JT1', LOAD_DATE=datetime.date(2023, 1, 24), SRC_IND='FIN_S3', LOADED_BY='BATCH'), Row(CL

[Row(CURRENT_WAREHOUSE()='MIM_WH', CURRENT_DATABASE()='D_FIN_DMA', CURRENT_SCHEMA()='FIN_DMA')]


In [4]:
import pandas as pd
snow_df_car = test_session.table("D_FIN_STG.FIN_STG.CAR_DTA")
snow_df_save = test_session.table("D_FIN_STG.FIN_STG.SAVERISK_DTA")
snow_df_txn = test_session.table("D_FIN_STG.FIN_STG.TRANS_DTA")

In [41]:
#!pip install "snowflake-connector-python[pandas]"

In [5]:
car=snow_df_car.to_pandas()
saverisk =snow_df_save.to_pandas()
txn = snow_df_txn.to_pandas()

In [6]:
txn.head()

,SENDER,RECEIVER,AMOUNT,TRANS_DATE,TYPE_TRAN,BANK,LOAD_DATE,SRC_IND,LOADED_BY
0,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,566826.0,2022-04-22,rtgs_out,BANK OF INDIA,2023-02-06,FIN_S3,BATCH
1,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,5767445.0,2022-03-31,rtgs_out,BANK OF INDIA,2023-02-06,FIN_S3,BATCH
2,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,5857318.0,2022-01-16,rtgs_out,INDIAN BANK,2023-02-06,FIN_S3,BATCH
3,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,5919445.0,2022-02-25,rtgs_out,IDBI BANK,2023-02-06,FIN_S3,BATCH
4,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,5979951.0,2022-03-08,rtgs_out,CITI BANK,2023-02-06,FIN_S3,BATCH


In [7]:
txn.shape

(2364483, 9)

In [8]:
import re
import jaro
import time
import random
import string
import rapidfuzz
import numpy as np
import pandas as pd
from rapidfuzz.process import cdist
from gensim.models import FastText

In [9]:
#!pip install gensim

In [10]:
def name_standardize(name):
    orig_name = str(name)
    name = name.lower()
    name = name + ' '
    opc = [' opc ', ' one person company ']
    llp = [' llp ', 'llp', ' limited liability partnership ', ' limited liablity partnership ']
    pvt = ['private limited', ' private l ', 'pvtltd', 'pvt.ltd.', ' p ltd', 'pvt lt', 'pvt l', ' li ', 'privatelimited', ' private ', ' i pvt ', ' pvt ', ' pvt. ', ' lt ', ' (p) ', ' l ', 'p l ']
    pub = [' public limited ', ' public ', ' limited ', ' limited. ', 'limited', 'limitd', ' limit', ' limi', ' ltd. ', ' ltd ', 'ltd', ' lim']
    misc = ['co op', 'co-op',  'coop', ' and ', ' & co. ', ' co. ', ' corp. ', ' corpn ', ' corp ', ' & co ' , ' co ', ' op ', ' group ', ' indian ', ' india ', '^india', ' indi ', ' ind ', ' sal ', ' jan ', ' feb ', ' mar ', ' apr ', ' may ', ' jun ', ' jul ', ' aug ', ' sep ', ' oct ', ' nov ', ' dec ', ' of ', ' a/c ', ' a c ', ' dividend ', ' dividen ', ' the ', '^the ', ' ac ', ' ji ', '�', ' rtgs ', ' neft ', ' imps ', ' mumbai ', ' new delhi ', ' delhi ', ' online ', ' collections ', ' collection ', ' inc ', ' ins ', ' ent ', ' insur ', ' n ', ' llc ', ' inc ', ' ins ', ' ent ', ' insur ', 'india', ' trans ', ' sup ', ' fin ', ' div ', ' litd ']
    salutation = ['^mr ', '^mrs ', '^ms ', '^miss ', '^m/s ', '^sri ', '^shri ', '^shree ', '^dr ']

    terms = opc + llp + pvt + pub + misc + salutation
    name = re.sub('[^a-zA-Z ]', ' ', name)
    for i in range(len(terms)):
        if terms[i] == ' (p) ':
            name = re.sub('\(.*?\)', ' ', name)
        else:
            name = re.sub(terms[i],' ',name)
    name = re.sub(' +', ' ', name)
    if name == ' ':
        name = orig_name
    name = name.split()
#     name.sort()
    return " ".join(name)

In [11]:
car['CLIENT_NAMES'] = car['CLIENT_NAMES'].astype('str')
txn['RECEIVER'] = txn['RECEIVER'].apply(lambda x: x if (isinstance(x, str)) else np.nan)
txn = txn.dropna(subset=['RECEIVER'], axis=0)

print(car.shape, txn.shape)

(11234, 4) (2362392, 9)


In [12]:
car.head()

,CLIENT_NAMES,LOAD_DATE,SRC_IND,LOADED_BY
0,,2023-01-24,FIN_S3,BATCH
1,**,2023-01-24,FIN_S3,BATCH
2,0HRI SWAMI SAMARTHA ICE PRODUCTS PRIVATE LTD,2023-01-24,FIN_S3,BATCH
3,3.R. TALWALKAR (STATES) LIMITED,2023-01-24,FIN_S3,BATCH
4,4CS (INDIA) PRIVATE LIMITED,2023-01-24,FIN_S3,BATCH


In [13]:
car_data_standardized = car['CLIENT_NAMES'].apply(name_standardize)
test_data_standardized = txn['RECEIVER'].apply(name_standardize)
car['std_name'] = car_data_standardized
txn['std_name'] = test_data_standardized
car = car.drop_duplicates()
print(car.shape)

(11234, 5)


In [14]:
print(car.index.max())
car = car.reset_index(drop=True)
print(car.index.max())

11233
11233


In [15]:
def exact_match(names_1,names_2):
    return set(names_1).intersection(names_2)

In [16]:
txn['matches'] = np.nan
txn = txn.drop_duplicates(subset=['RECEIVER'])
print('Data after removing duplicates: ', txn.shape)
exact_set = exact_match(car['std_name'], txn['std_name'])
exact_matches = txn['std_name'].isin(exact_set)
exact_match_index = exact_matches.index[exact_matches == True].tolist()
txn.loc[exact_match_index, 'matches'] = txn.loc[exact_match_index, 'std_name']
data_found_exact_match = txn[txn['matches'].notnull()]
print(f"Found {txn[txn['matches'].notnull()].shape[0]} exact matches")
print(f"Remaining data to be matched {txn[txn['matches'].isnull()].shape[0]}")
car.shape[0]

Data after removing duplicates:  (2873, 11)
Found 231 exact matches
Remaining data to be matched 2642


11234

In [17]:
txn[txn['matches'].notnull()]['RECEIVER'].nunique()

231

In [18]:
def optimized_levenshtein_rapidfuzz(threshold, data):
    ratios = cdist(data['std_name'], car['std_name'], score_cutoff=threshold, workers=-1)
    max_indexes = ratios.argmax(axis=1)
    test_indexes = np.arange(max_indexes.shape[0])
    scores = ratios.max(axis=1)
    scores_index = np.where(scores >= threshold)
    max_indexes_filtered, test_indexes_filtered = max_indexes[scores_index], test_indexes[scores_index]
    data.loc[test_indexes_filtered, 'matches'] = car.iloc[max_indexes_filtered, 0].values
    return data


def optimized_levenshtein(data, threshold, name_1, col_to_return=0):
    ratios = data['std_name'].apply(rapidfuzz.fuzz.ratio,args=(name_1,))
    if ratios.max() >= threshold:
        return data.iloc[ratios.idxmax(), col_to_return], ratios.max()

In [19]:
remain_cases = txn[txn['matches'].isnull()]
remain_cases = remain_cases.drop_duplicates(subset=['RECEIVER'])
remain_cases = remain_cases.reset_index(drop=True)
print("Remaining data to be matched", remain_cases.shape[0])
remain_cases['matches'] = remain_cases.apply(lambda x: optimized_levenshtein(car, 80, x['std_name']), axis=1)
remain_cases['matches'], remain_cases['match_score'] = remain_cases.matches.str
print("Remaining data to be matched", remain_cases[remain_cases['matches'].isnull()].shape[0])

Remaining data to be matched 2642
Remaining data to be matched 2154


C:\Users\10663901\AppData\Local\Temp\ipykernel_12556\1507119391.py:6: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  remain_cases['matches'], remain_cases['match_score'] = remain_cases.matches.str


In [20]:
remain_cases[remain_cases['matches'].notnull()]

,SENDER,RECEIVER,AMOUNT,TRANS_DATE,TYPE_TRAN,BANK,LOAD_DATE,SRC_IND,LOADED_BY,std_name,matches,match_score
3,KIRTI PRAKASH CHANDANE,ABHISHEK LALMAN PRAJAPATI,273.73,2022-03-11,upi_out,ICICI BANK LIMITED,2023-02-06,FIN_S3,BATCH,abhishek lalman prajapati,ABHISHEK LAccAN PRaJvPATz,84.000000
5,KIRTI PRAKASH CHANDANE,AIRTEL,1026.89,2022-03-04,upi_out,CITI BANK,2023-02-06,FIN_S3,BATCH,airtel,vIRTEL,83.333333
7,KIRTI PRAKASH CHANDANE,AMAZON,1.00,2022-02-28,upi_out,CITI BANK,2023-02-06,FIN_S3,BATCH,amazon,AMAZONuA,85.714286
8,KIRTI PRAKASH CHANDANE,AMAZON PAY,1147.72,2022-05-22,upi_out,AXIS BANK,2023-02-06,FIN_S3,BATCH,amazon pay,AMAZuN PAp,80.000000
9,KIRTI PRAKASH CHANDANE,AMAZONPA,1.00,2021-12-17,upi_out,ICICI BANK LIMITED,2023-02-06,FIN_S3,BATCH,amazonpa,AMAZONuA,87.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
2579,ARVIND CAULAGI,ARVIND CAULAGI,1.00,2021-12-04,upi_out,AXIS BANK,2023-02-06,FIN_S3,BATCH,arvind caulagi,ARVIND MALI,80.000000
2592,ARVIND CAULAGI,INCAN MUTUAL BENEFIT LIMITED,1484.39,2022-03-09,upi_out,ICICI BANK LIMITED,2023-02-06,FIN_S3,BATCH,incan mutual benefit,RTC MUTUAL BENEFIT LIMITED,84.210526
2623,ASHISH SANJAY GIRI,RAKESH FINANCE LIMITED,1252.62,2022-04-29,upi_out,AU SMALL FINANCE BANK LIMITED,2023-02-06,FIN_S3,BATCH,rakesh finance,MANKESHWAR FINANCE PRIVATE LIMITED,81.250000
2628,ASHRAF ALI,ASHRAF ALI,1.00,2022-03-19,upi_out,DCB BANK LIMITED,2023-02-06,FIN_S3,BATCH,ashraf ali,ASHIF ALI,84.210526


In [21]:
final_found = pd.concat((txn[txn['matches'].notnull()].drop_duplicates(subset=['RECEIVER']), remain_cases[remain_cases['matches'].notnull()]), axis=0)
final_found

,SENDER,RECEIVER,AMOUNT,TRANS_DATE,TYPE_TRAN,BANK,LOAD_DATE,SRC_IND,LOADED_BY,std_name,matches,match_score
0,KIRAN KUMAR TELIKEPALLI,SILVEX EXPORTS PVT LTD,566826.00,2022-04-22,rtgs_out,BANK OF INDIA,2023-02-06,FIN_S3,BATCH,silvex exports,silvex exports,NaN
23,KIRAN KUMAR TELIKEPALLI,SMARDOCE SOLUTIONS PVT LTD,1263575.00,2022-01-23,rtgs_out,HDFC BANK,2023-02-06,FIN_S3,BATCH,smardoce solutions,smardoce solutions,NaN
143,KIRAN KUMAR TELIKEPALLI,SONALI JINDAL,10927693.00,2022-03-21,rtgs_out,CANARA BANK,2023-02-06,FIN_S3,BATCH,sonali jindal,sonali jindal,NaN
229,KIRAN KUMAR TELIKEPALLI,SWATI SINGH,1075427.00,2021-12-13,rtgs_out,STATE BANK OF INDIA,2023-02-06,FIN_S3,BATCH,swati singh,swati singh,NaN
365,KIRAN KUMAR TELIKEPALLI,TEAMWORKS MANPOWER CONSULTING,1006461.00,2022-03-15,rtgs_out,HDFC BANK,2023-02-06,FIN_S3,BATCH,teamworks manpower consulting,teamworks manpower consulting,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2579,ARVIND CAULAGI,ARVIND CAULAGI,1.00,2021-12-04,upi_out,AXIS BANK,2023-02-06,FIN_S3,BATCH,arvind caulagi,ARVIND MALI,80.000000
2592,ARVIND CAULAGI,INCAN MUTUAL BENEFIT LIMITED,1484.39,2022-03-09,upi_out,ICICI BANK LIMITED,2023-02-06,FIN_S3,BATCH,incan mutual benefit,RTC MUTUAL BENEFIT LIMITED,84.210526
2623,ASHISH SANJAY GIRI,RAKESH FINANCE LIMITED,1252.62,2022-04-29,upi_out,AU SMALL FINANCE BANK LIMITED,2023-02-06,FIN_S3,BATCH,rakesh finance,MANKESHWAR FINANCE PRIVATE LIMITED,81.250000
2628,ASHRAF ALI,ASHRAF ALI,1.00,2022-03-19,upi_out,DCB BANK LIMITED,2023-02-06,FIN_S3,BATCH,ashraf ali,ASHIF ALI,84.210526


In [22]:
data_remain_saverisk = remain_cases[remain_cases['matches'].isnull()]
data_remain_saverisk

,SENDER,RECEIVER,AMOUNT,TRANS_DATE,TYPE_TRAN,BANK,LOAD_DATE,SRC_IND,LOADED_BY,std_name,matches,match_score
0,KIRAN KUMAR TELIKEPALLI,UCO BANK,2628299.00,2022-02-25,rtgs_out,INDUSIND BANK,2023-02-06,FIN_S3,BATCH,uco bank,None,NaN
1,KIRAN KUMAR TELIKEPALLI,service duty on purchase,554690.00,2022-03-08,rtgs_out,HDFC BANK,2023-02-06,FIN_S3,BATCH,service duty on purchase,None,NaN
2,KIRAN KUMAR TELIKEPALLI,tax towards FD,364320.00,2022-04-07,rtgs_out,UNION BANK OF INDIA,2023-02-06,FIN_S3,BATCH,tax towards fd,None,NaN
4,KIRTI PRAKASH CHANDANE,AGGREPAY PAYMENTS SOLUTION PVT LTD,1669.60,2021-10-17,upi_out,YES BANK,2023-02-06,FIN_S3,BATCH,aggrepay payments solution,None,NaN
6,KIRTI PRAKASH CHANDANE,AIRTEL PAYMENTS BANK LIMITED,1030.30,2022-04-01,upi_out,ICICI BANK LIMITED,2023-02-06,FIN_S3,BATCH,airtel payments bank,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2636,ASMI ANBU,ASMI ANBU,1.00,2022-03-19,upi_out,BASSEIN CATHOLIC COOPERATIVE BANK LIMITED,2023-02-06,FIN_S3,BATCH,asmi anbu,None,NaN
2637,ASMI ANBU,GLOBAL GELATINES LTD,1472.28,2022-03-02,upi_out,AXIS BANK,2023-02-06,FIN_S3,BATCH,global gelatines,None,NaN
2639,ATUL SINGH,DEE COAL BOXES LIMITED,910.87,2022-03-22,upi_out,PAYTM PAYMENTS BANK LTD,2023-02-06,FIN_S3,BATCH,dee coal boxes,None,NaN
2640,AVANT FINWISE PRIVATE LIMITED,AVANT FINWISE PRIVATE LIMITED,100.00,2022-05-21,NEFT_out,STATE BANK OF INDIA,2023-02-06,FIN_S3,BATCH,avant finwise,None,NaN


In [23]:
saverisk_standardized = saverisk['COMPANY'].apply(name_standardize)
saverisk['std_name'] = saverisk_standardized
saverisk = saverisk.drop_duplicates()
print(saverisk.shape)

print(saverisk.index.max())
saverisk = saverisk.reset_index(drop=True)
print(saverisk.index.max())

(129087, 8)
129086
129086


In [24]:
exact_set = exact_match(saverisk['std_name'], data_remain_saverisk['std_name'])
exact_matches = data_remain_saverisk['std_name'].isin(exact_set)
exact_match_index = exact_matches.index[exact_matches == True].tolist()
data_remain_saverisk.loc[exact_match_index, 'matches'] = data_remain_saverisk.loc[exact_match_index, 'std_name']
data_found_exact_match = data_remain_saverisk[data_remain_saverisk['matches'].notnull()]
print(f"Found {data_remain_saverisk[data_remain_saverisk['matches'].notnull()].shape[0]} exact matches")
print(f"Remaining data to be matched {data_remain_saverisk[data_remain_saverisk['matches'].isnull()].shape[0]}")
saverisk.shape[0]

Found 424 exact matches
Remaining data to be matched 1730


129087

In [25]:
data_saverisk_scrub2 = data_remain_saverisk[data_remain_saverisk['matches'].isnull()]
print("Remaining data to be matched", data_saverisk_scrub2.shape[0])
data_saverisk_scrub2['matches'] = data_saverisk_scrub2.apply(lambda x: optimized_levenshtein(saverisk, 80, x['std_name'], 1), axis=1)
data_saverisk_scrub2['matches'], data_saverisk_scrub2['match_score'] = data_saverisk_scrub2.matches.str
print("Remaining data to be matched", data_saverisk_scrub2[data_saverisk_scrub2['matches'].isnull()].shape[0])

Remaining data to be matched 1730
Remaining data to be matched 1362


C:\Users\10663901\AppData\Local\Temp\ipykernel_12556\211102786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_saverisk_scrub2['matches'] = data_saverisk_scrub2.apply(lambda x: optimized_levenshtein(saverisk, 80, x['std_name'], 1), axis=1)
C:\Users\10663901\AppData\Local\Temp\ipykernel_12556\211102786.py:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  data_saverisk_scrub2['matches'], data_saverisk_scrub2['match_score'] = data_saverisk_scrub2.matches.str
C:\Users\10663901\AppData\Local\Temp\ipykernel_12556\211102786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [26]:
final_result = pd.concat((data_remain_saverisk[data_remain_saverisk['matches'].notnull()], data_saverisk_scrub2[data_saverisk_scrub2['matches'].notnull()], final_found), axis=0)
final_result.shape

(1511, 12)

In [27]:
final_result.to_csv('name_matching_results_0702.csv')

In [ ]:
nameMatching_df.columns=['SENDER_NAME','RECEIVER_NAME','AMOUNT_VAL','TRANS_DATE','TYPE_TRAN','BANK_NAME','STD_NAME','MATCH_NAME']
nameMatching_df3 = test_session.write_pandas(nameMatching_df,"D_FIN_DMA.FIN_DMA.NAME_MATCHING_RESULTS",auto_create_table=False, overwrite=True)
